# 第 4 章：概率与统计（Probability & Statistics for Runtime Inference）

本章重点：

- 量化和剪枝中的“分布视角”
- 蒸馏和温度缩放中的 KL/交叉熵
- 通过分位数、直方图选择 clipping 阈值


## 4.1 输出分布与 KL 散度

### 4.1.1 交叉熵与 KL

给定真实分布 $p$ 与模型分布 $q$：

- 交叉熵：

$$
H(p, q) = - \sum_i p_i \log q_i
$$


- KL 散度：

$$
\text{KL}(p\|q) = \sum_i p_i \log \frac{p_i}{q_i}
$$


二者关系：

$$
H(p, q) = H(p) + \text{KL}(p\|q)
$$


其中 $H(p)$ 与参数无关，优化时常直接最小化交叉熵。

在蒸馏中，teacher 分布 $p^T$，student 分布 $p^S$：

$$
L_{\text{KD}} = \text{KL}(p^T \| p^S)
$$


反映 student 在多大程度上“模仿”了 teacher 的输出分布。

**工程意义**
【场景 A：蒸馏 student 接 teacher 输出分布】
Teacher 输出 logits：
```block
cat: 0.7, dog: 0.25, car: 0.05

```
Student 输出 logits：
```block
cat: 0.5, dog: 0.45, car: 0.05
```
即使 student 的 argmax 仍正确（cat），
但是 teacher 的“信心结构”不同，KL 会惩罚：

teacher 认为 second-most 类别是 dog

student 却认为 dog 可能性太高（过度自信）
部署意义：
这样的 student 在真实世界 distribution shift 时更不稳健

【场景 B：量化后 logits 改变 → KL 变大 → 精度掉】
部署 INT8 模型时，logits 会变得更粗糙，例如：
FP32 teacher：
```block
cat: 2.10, dog: 1.70, car: -3.25
```
INT8 student：
```block
cat: 2.0, dog: 1.9, car: -3.0
```
Logit 的微小差别在 softmax 后会被放大：

原本 teacher 中 cat ≫ dog

INT8 后 cat ≈ dog

此时 KL(p_T || p_S) 会升高 → 精度下降
这就是为什么模型量化前要蒸馏（QAT 里最关键部分）。



为什么说“模型量化前要蒸馏”？这不是严格的因果关系，而是一个工程上常用的实践：蒸馏能减少量化引入的性能下降，提升量化后模型的鲁棒性和准确率。要点总结：

- 蒸馏不会必然导致量化成功，而是提供一种更鲁棒的训练信号，使得模型对量化扰动不那么敏感，从而在实践中经常减少精度损失。
- 机制（为什么有用）：
    - 软标签（teacher 的 soft logits）携带“暗知识”（类别间相对概率），比硬标签更丰富，能让 student 学到类别关系，而不是只学决策边界。
    - 温度平滑（T>1）让概率分布更平缓，学生学到的边界更平滑，面对量化引入的噪声时更稳定。
    - 蒸馏倾向于减小 logits 的极端差异、平滑输出分布，从而降低对动态范围和精度的依赖，利于后续的 PTQ（post-training quant）。
    - 如果在量化感知训练（QAT）中同时用 KD，会让量化仿真（fake-quant）下的 student 更快收敛，减少量化误差的放大。
- 两种常用工作流：
    1. 先蒸馏（FP32 student 学到 teacher 的分布），再做 PTQ —— 简单且常有效。
    2. 在 QAT 中同时做 KD（边做 fake-quant 边用 teacher 的 soft-targets）——最稳健，但训练成本高。
- 常用公式（简短）：
    L = (1 - α) * CE(y_true, p_S) + α * T^2 * KL(soft_T(p_T) || soft_T(p_S))
    建议超参经验值：T≈2–4，α≈0.3–0.7（视任务与数据量调整）。
- 何时不必蒸馏：
    - 你能做充分的 QAT（有足够计算与数据），QAT 本身就能恢复精度；
    - 模型/任务对量化不敏感或 PTQ 已经无明显精度下降；
    - 资源或时间不足以做蒸馏（可以先尝试简单 PTQ 验证）。

结论：蒸馏不是量化的必要条件，但在资源有限、想用 PTQ 或想降低量化带来精度损失时，蒸馏是一个有效的工具；若能在 QAT 中结合 KD，通常能得到更好的量化后性能。

是所有模型都要蒸馏？

不是。

以下模型 **不需要蒸馏也能量化很稳**：

- ResNet / MobileNet / EfficientNet（视觉 backbone）

- UNet（分割）

- 低深度 MLP

- 这些模型的 logits 结构比较“鲁棒”，量化噪声不容易破坏排序结构。

ℹ️ 工程经验

- CNN 很好量化
- Transformer 很难量化
- 尤其是 attention / FFN logits 对量化非常敏感

所以：

- CNN → PTQ 就足够稳定
- Transformer → QAT + 蒸馏 基本是 industry standard

**什么时候“蒸馏 ≈ 强制项”？**
| 模型类型                      | PTQ 精度掉多少 | 是否建议蒸馏    |
| ------------------------- | --------- | --------- |
| ResNet-50                 | < 0.5%    | 不需要       |
| YOLOv5                    | < 1%      | 不需要（但会更好） |
| UNet                      | < 0.2%    | 不需要       |
| Transformer encoder       | 2–6%      | ✔️ 建议蒸馏   |
| Transformer decoder (LLM) | 5–15%     | ✔️ 强制     |
| 多任务统一模型                   | 3–10%     | ✔️ 强制     |
| 多 modality 模型             | 5–20%     | ✔️ 强制     |

❗ 为什么 Transformer 类型需要蒸馏？

因为它们：

- logits margin 小

- attention 分布脆弱

- feedforward 权重 heavy-tail

- LayerNorm 导致 activation variance 高

- softmax 超敏感

- 量化后 logits 常常会“排序反转”（rank flip），这是纯 PTQ 无法修复的，而蒸馏能主动学习去抵抗 rank flip。

**最终总结：你应该怎么判断？**
✔️ 如果量化后 KL(p_T || p_S) 上升很快 → 建议蒸馏
✔️ 如果量化后 accuracy 掉 3% 以上 → 建议蒸馏
✔️ 如果是 Transformer 模型 → 几乎必做蒸馏
✔️ 如果是安全/自动驾驶模型 → 强烈建议蒸馏（因为风险高）


### 4.1.2 温度 T：真实部署中的作用
softmax with temperature：
$$ p_i = \frac{e^{z_i / T}}{\sum_j e^{z_j / T}} $$

数学意义

- 𝑇 > 1 分布更平滑
- 𝑇 < 1 分布更尖锐

**工程例子**：Teacher 的 logits 非常尖锐
某 traffic-light 状态检测模型：

Teacher logits：
``` block
red: 9.2, yellow: -2.1, green: -5.4
```
softmax 后几乎是 one-hot，信息极度稀疏。

Student 很难学到 teacher 的“relative probability structure”。

使用 T=2：
```block
red: 9.2/2 = 4.6
yellow: -2.1/2 = -1.05
green: -5.4/2 = -2.7

```
softmax 变得更平滑（暗知识被放大）。
**部署场景**

当你把 INT8 模型部署到车端：

- 在 分布 shift（雨夜、逆光、dirty windshield）下

- student 模型如果来自 高温蒸馏，会保持更稳定的 logits 结构

- 分类错误率显著下降

这是实际模型部署中非常常见的效果

## 4.2 量化中的统计视角：直方图与分位数
### 4.2.1 分位数激活数学 

设 𝑥 为 activations：
$$\alpha=quantile_p​(∣x∣)$$
选择 clipping 范围 [−α,α]。

### 4.2.2 为什么不使用 max
假设激活分布如下：

- 98% 的激活都在 [-0.5, 0.5]

- 但 max 值可能是 7.5（极端 outlier）

如果用 max 作为量化区间 
- INT8 对称量化时，量化范围为：$[-\alpha,\, \alpha] = [-x_{\max},\, x_{\max}] = [-7.5,\, 7.5]$, 于是 scale：$\text{scale} = \alpha/127 = 7.5/127 \approx 0.05905$ 则int8量化为
$$
x_{\text{int8}} = \text{round}\left( \frac{x_{\text{fp32}}}{\text{scale}} \right)
$$
- 98% 的激活 [-0.5,0.5] 占整个范围 [-7.5,7.5] 的多少比例？0.5/7.5 = 1/15.也就是： 98% 的激活只占整个动态范围的 1/15。 其量化分配的 bin 只有原本可用 bin 数的 1/15。

也就是说：

INT8 有 255 个可用 bin

但 98% 的激活只能使用：255/15 ~ 17bin

**工程例子**
假设某一层激活：
```block
[-0.4, -0.38, -0.35, ..., 0.33, 0.4]   (98%)
[7.8]                                  (rare outlier)
```

max = 7.8 → scale = 7.8/127 = 0.0614
这样大部分激活变成
```block
0.4 -> q = round(0.4 / 0.0614) = 6    (only 6/127!)
```

量化后的信息严重损失。使用99.9%分位数
假设 99.9% 分位数是 0.45

scale = 0.45/127 = 0.00354
激活的量化 bin 非常细，保留了大量信息。


### 4.2.3 量化误差propagation
**用max的后果**： 量化误差的标准差扩大 15 倍。

在均匀量化中，最关键因素是 bin width，也叫量化步长 $\Delta$：

$$
\Delta = \text{scale}
$$

如果用 $\text{max}=7.5$ 作为阈值：

$$
\Delta_{\text{max}} = 0.05905
$$

如果不用 max，而使用真实有效范围 0.5：

$$
\Delta_{\text{good}} = \frac{0.5}{127} = 0.003937
$$

对比：

$$
\frac{\Delta_{\text{max}}}{\Delta_{\text{good}}} = \frac{0.05905}{0.003937} \approx 15
$$

量化误差标准差近似为：

$$
\sigma_{\text{quant}} \approx \frac{\Delta}{\sqrt{12}}
$$

所以：

$$
\sigma_{\text{bad}} \approx 15 \times \sigma_{\text{good}}
$$

误差大 15 倍 → 进入网络后层层扩散 → 性能下降。



### 4.2.3  KL-based 最优裁剪阈值 α 的整体思路

整体思路（TensorRT / TF-Lite 核心算法）

**核心目标**：  
为每一层激活/权重找到一个最优的 clipping 范围 $[-α, α]$，使得量化误差（用 KL 散度衡量）最小。

#### KL 散度最小化校准算法
对每个候选 α 执行： 
1. 从激活样本构建直方图分布

收集激活样本 $ x_1, \dots, x_N $，在固定区间 $[a, b]$ 上划分为 $ K $ 个等宽 bin：

- 第 $ i $ 个 bin 区间为 $ [b_i, b_{i+1}) $
- 计数为 $ c_i $

概率：
$$
p(i) = \frac{c_i}{\sum_{k=1}^K c_k}, \quad i = 1,\dots,K
$$

2. 遍历各个候选阈值（用 bin 索引 $ T $ 表示）

通常我们不直接用实数 $ \alpha $，而用某个 bin 索引 $ T $ 作为阈值。

比如只考虑 histogram 的前 $ T $ 个 bin，把 $ [T+1, \dots, K] $ 的质量折叠进第 $ T $ 个 bin。

构造参考分布 $ p_{\text{ref}}^{(T)} $：

$$
p_{\text{ref}}^{(T)}(i) = 
\begin{cases}
p(i),  & 1 \le i < T \\[4pt]
p(T) + \displaystyle\sum_{k=T+1}^K p(k),  & i = T \\[8pt]
0,  & i > T
\end{cases}
$$

归一化一次：
$$
p_{\text{ref}}^{(T)}(i) \leftarrow \frac{p_{\text{ref}}^{(T)}(i)}{\sum_{k=1}^T p_{\text{ref}}^{(T)}(k)}
$$

3. 把 $ T $ 个 bin 压缩成 $ Q $ 个 quant bin

令 $ Q $ 为量化级数（INT8 对称时 $ Q=255 $，INT8 非对称时 $ Q=256 $，INT4 时 $ Q=15 $）。

我们把 $ 1,\dots,T $ 分成 $ Q $ 组：
$$
G_1^{(T)}, G_2^{(T)}, \dots, G_Q^{(T)}
$$

每个组内包含一段连续的 bin 索引（通常按“合并相邻”规则实现）。

对每个组求和得到量化分布：
$$
q^{(T)}(j) = \sum_{i \in G_j^{(T)}} p_{\text{ref}}^{(T)}(i), \quad j=1,\dots,Q
$$

4. 把量化后的 $ Q $ 个 bin 再展开回 $ T $ 个 bin：得到近似分布 $ \hat{p}^{(T)} $

最简单的展开方式（组内均匀）：
$$
\hat{p}^{(T)}(i) = \frac{q^{(T)}(j)}{|i \in G_j^{(T)})}{|G_j^{(T)}|}, \quad \forall i \in G_j^{(T)}
$$

这样 $ p_{\text{ref}}^{(T)} $ 和 $ \hat{p}^{(T)} $ 都是维度 $ T $ 的分布。

5. 计算 KL：得到每个阈值 $ T $ 对应的 KL

$$
\text{KL}(T) = \text{KL}\bigl(p_{\text{ref}}^{(T)} \parallel \hat{p}^{(T)}\bigr)
= \sum_{i=1}^T p_{\text{ref}}^{(T)}(i) \log \frac{p_{\text{ref}}^{(T)}(i)}{\hat{p}^{(T)}(i) + \epsilon}
$$

最后选择：
$$
T^* = \arg\min_T \text{KL}(T)
$$

得到最优阈值对应的 bin，再映射回实数域得到 $ \alpha^* $。

In [ ]:
## ChatGPT implementation of KL-based Threshold Selection for Quantization Calibration
import torch

def kl_calibration_hist(samples, num_bins=2048, num_quant_bins=255):
    # 1. 构建高精度直方图
    vmin, vmax = samples.min().item(), samples.max().item()
    hist = torch.histc(samples, bins=num_bins, min=vmin, max=vmax)
    hist = hist.to(torch.float64)
    hist_sum = hist.sum()
    p = hist / hist_sum   # 原始概率分布

    # 候选阈值用 bin 索引表示：从某个最小 T 到 num_bins-1
    # 例如跳过最左边第一个，避免太窄
    candidate_T = torch.arange(num_quant_bins, num_bins)

    kl_values = []
    for T in candidate_T:
        T = int(T.item())

        # ---- Step 1: 构造参考分布 p_ref^{(T)} ----
        ref = p.clone()
        # 将 [T+1, ..., K] 的质量折叠到第 T 个 bin
        tail = ref[T+1:].sum()
        ref[T] += tail
        ref[T+1:] = 0.0

        # 截取前 T+1 个 bin 作为有效区间
        ref = ref[:T+1]
        ref = ref / ref.sum()

        # ---- Step 2: 将 T+1 个 bin 压缩为 num_quant_bins 个组 ----
        n = ref.numel()
        if n <= num_quant_bins:
            # bin 数太少，不适合做 KL 校准
            continue

        # group_boundaries: 把 [0, n) 均分为 num_quant_bins 组
        boundaries = torch.linspace(0, n, steps=num_quant_bins+1, dtype=torch.long)
        q = torch.zeros_like(ref)

        # 压缩：每组的质量合并为一个 "quant bin"
        quant_hist = torch.zeros(num_quant_bins, dtype=torch.float64)
        for j in range(num_quant_bins):
            start = boundaries[j].item()
            end   = boundaries[j+1].item()
            if start >= end:
                continue
            quant_hist[j] = ref[start:end].sum()

        # ---- Step 3: 展开：把 quant_hist 再均匀分配回对应组 ----
        approx = torch.zeros_like(ref)
        for j in range(num_quant_bins):
            start = boundaries[j].item()
            end   = boundaries[j+1].item()
            if start >= end:
                continue
            if quant_hist[j] > 0:
                approx[start:end] = quant_hist[j] / (end - start)

        # ---- Step 4: 计算 KL(ref || approx) ----
        eps = 1e-12
        ref_safe   = ref + eps
        approx_safe = approx + eps
        kl = torch.sum(ref_safe * torch.log(ref_safe / approx_safe))
        kl_values.append((T, kl.item()))

    # 选 KL 最小的阈值
    if not kl_values:
        raise RuntimeError("No valid candidate thresholds for KL calibration.")

    T_best, kl_min = min(kl_values, key=lambda x: x[1])
    # 对应的 alpha（实数阈值）
    bin_edges = torch.linspace(vmin, vmax, steps=num_bins+1)
    alpha_best = bin_edges[T_best+1].item()

    return alpha_best, kl_min, kl_values


torch.manual_seed(0)
samples = torch.cat([
    torch.randn(100_000) * 1.0,        # 主体
    torch.randn(200) * 0.5 + 6.0,      # outliers
])

alpha_best, kl_min, kv = kl_calibration_hist(samples)
print("best alpha:", alpha_best, "min KL:", kl_min)

best alpha: 6.784961700439453 min KL: 0.007733591698964529


## 4.3 统计稳健性（Robust Statistics）

在推理部署中，你需要考虑：

- 上游数据分布是否会发生漂移（distribution shift）
- 是否存在噪声、异常值（outliers）
- 对不同层的激活分布要有宏观认知：
  - 是否近似高斯
  - 是否有长尾

有些技术（例如 robust norm、Huber 损失）可以缓解 outlier 对参数/统计量的影响。  
在这里你不需要深入推导，只需要知道：

- 许多可以“抑制 outlier”的方法本质上来自稳健统计理论
- 在量化/clip/归一化中，使用分位数而不是 max/min 也是一种稳健统计实践

【工程例子】
例 1：激活分布的长尾导致部署事故

真实工程观察：
某些层（特别是 LayerNorm 前或 attention Q/K/V）：

激活呈 长尾分布

- 10⁻³ 的 outlier 会扩大量化 scale ×3~5

- 一下子毁掉该层 FP32 → INT8 信息

解决方案：

- 分位数（percentile）

- Huber loss（训练阶段）

- Log-activation（某些架构）
---
例 2：distribution shift（雨天/逆光）让激活偏移

量化时收集的统计往往来自：

- 训练数据

- 或清洁白天数据

实际部署：

- 夜间 headlights 让 feature 累计偏置

- 导致偏移超出原本的量化范围

- Activation clipping 大量触发 → 精度下降

解决方案：

- 用 rolling window 动态校准（在线校准）

- 或使用更鲁棒的 scale 更新方法（EMA）
---
例 3：权重分布的 heavy-tail（例如 GPT）让量化困难

自回归模型的某些 FFN 层：

- 权重呈现 heavy-tail

- INT8 scale 很难设置

- 即使 0.1% 的过大 weight 也会 dominate scale

- 把所有正常权重全挤压到小 bin 内

解决方案：

- GPTQ（Gauss–Newton or Gradient-based Post-Training Quantization）

- AWQ (Activation-aware quantization)

- per-channel quantization

- 分位数权重剪裁（0.1% percentile）

## 4.4 本章总结
| 概念                  | 工程影响                                     |
| ------------------- | ---------------------------------------- |
| KL / 交叉熵            | 衡量 student 了解 teacher 多少；量化后 logits 变化大小 |
| Temperature         | 控制 soft label 平滑度；影响蒸馏成功率                |
| Percentile clipping | 抑制 outlier，显著提升 INT8 精度                  |
| KL-based threshold  | 用信息论方式自动寻找最佳 quantization range          |
| Robust statistics   | 解决长尾、噪声、分布漂移；对部署至关重要                     |


## 4.6 量化核心公式

### 4.6.1 Softmax with Temperature（带温度系数的 Softmax）

标准 softmax：
$$
p_i = \frac{e^{z_i}}{\sum_j e^{z_j}}
$$

带温度 T 的 softmax：
$$
p_i = \frac{e^{z_i / T}}{\sum_j e^{z_j / T}}
$$

其中：
- $ z_i $：logit
- $ T > 1 $：分布变平滑（soft labels）
- $ T < 1 $：分布变尖锐

### 4.6.2 KL 散度 & 交叉熵

给定 teacher 分布 $p^T$ 和 student 分布 $p^S$：

KL 散度（常用作蒸馏损失）：
$$
\text{KL}(p^T \parallel p^S) = \sum_i p_i^T \log \frac{p_i^T}{p_i^S}
$$

交叉熵：
$$
H(p, q) = -\sum_i p_i \log q_i
$$

关系式：
$$
H(p, q) = H(p) + \text{KL}(p \parallel q)
$$

### 4.6.3 量化前后的 KL 变化（核心公式）

量化后 student logits $z^S$ 因为 scale 和 rounding 发生变化，使得 softmax 后分布不同：
$$
p_i^S = \frac{e^{z_i^S}}{\sum_j e^{z_j^S}}
$$

如果量化破坏了 logits 的排序结构（relative margins）：
$$
\text{KL}(p^T \parallel p^S) \uparrow \quad \text{（导致预测精度下降）}
$$

### 4.6.4  蒸馏损失（QAT 常用）

量化训练中最常用的蒸馏损失（带温度）：
$$
\mathcal{L}_{\text{KD}} = T^2 \cdot \text{KL}\left( \text{softmax}\left(\frac{z^T}{T}\right) \parallel \text{softmax}\left(\frac{z^S}{T}\right) \right)
$$

其中 $T^2$ 是温度缩放校正因子（保证梯度幅度不变）。

### 4.6.5 分位数裁剪（Percentile Clipping）

激活 $x$ 的绝对值分位数：
$$
\alpha = \text{quantile}_p(|x|)
$$

量化范围：
$$
[-\alpha, \alpha]
$$

常见取法：p = 99.9%、99.99%、99.999% 等

### 4.6.6 KL-based 阈值选择（TensorRT / TF-Lite 常见做法）

给定原始直方图分布 $p(x)$ 和量化重建分布 $q(x)$，最佳阈值 $\alpha^*$ 满足：
$$
\alpha^* = \arg\min_{\alpha} \text{KL}\left(p(x) \parallel q_{\alpha}(x)\right)
$$

直接最小化重建分布和原始分布的 KL 散度，自动得到最优 clipping 范围。